In [8]:
def format_logs(log_lines):
    result = []
    stack = []  # 用来记录层级缩进

    i = 0
    while i < len(log_lines):
        line = log_lines[i].strip()
        if "输入" in line:
            # 当前模块名，例如 "Embedding.forward"
            module = line.split("] ")[1].split(" 输入:")[0]

            # 检查下一行是不是对应的输出
            if (
                i + 1 < len(log_lines)
                and module in log_lines[i + 1]
                and "输出" in log_lines[i + 1]
            ):
                indent = "    " * len(stack)
                inp = line.split("输入:")[1].strip()
                out = log_lines[i + 1].split("输出:")[1].strip()
                result.append(f"{indent}[LOG] {module} 输入:{inp} 输出:{out}")
                i += 2
                continue
            else:
                indent = "    " * len(stack)
                result.append(indent + line)
                stack.append(module)
        elif "输出" in line:
            # 输出但没有输入配对
            indent = "    " * (len(stack) - 1 if stack else 0)
            result.append(indent + line)
            if stack:
                stack.pop()
        else:
            # 其他 log 原样缩进
            indent = "    " * len(stack)
            result.append(indent + line)
        i += 1
    # 删除`[LOG]`
    result = [line.replace("[LOG] ", "") for line in result]
    return "\n".join(result)


if __name__ == "__main__":
    with open("model.log", "r") as f:
        log_lines = f.readlines()

    formatted = format_logs(log_lines)
    with open("formatted_log.txt", "w") as f:
        f.write(formatted)
    print(formatted)


EncoderDecoder.forward 输入: [torch.Size([16, 128]), torch.Size([16, 128]), torch.Size([16, 1, 1, 128])]
    Sequential.forward 输入: [torch.Size([16, 128])]
        Embeddings.forward 输入: [torch.Size([16, 128])]
            Embedding.forward 输入:[torch.Size([16, 128])] 输出:torch.Size([16, 128, 768])
        Embeddings.forward 输出: torch.Size([16, 128, 768])
        PositionalEncoding.forward 输入: [torch.Size([16, 128, 768])]
            Dropout.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
        PositionalEncoding.forward 输出: torch.Size([16, 128, 768])
    Sequential.forward 输出: torch.Size([16, 128, 768])
    Encoder.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
        EncoderLayer.forward 输入: [torch.Size([16, 128, 768]), torch.Size([16, 1, 1, 128])]
            SublayerConnection.forward 输入: [torch.Size([16, 128, 768])]
                LayerNorm.forward 输入:[torch.Size([16, 128, 768])] 输出:torch.Size([16, 128, 768])
                MultiHeaded